**LetsGrowMore Data Science Virtual Internship**

Author = **Rupak Dey**

Level 03 = **Advanced Level Task**

Task 02 = **Next Word Prediction**

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
path = '/content/drive/MyDrive/Colab Notebooks/1661-0.txt'
text = open(path).read().lower()

In [3]:
#replace unnecessary stuff with space
text = text.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','') 

In [4]:
data = text.split()
data = ' '.join(data)

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[159, 4841, 1, 956, 5, 122, 32, 44, 548, 2007, 4842, 1035, 13, 21, 1]

In [6]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

13136


In [7]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  102504


array([[ 159, 4841,    1,  956],
       [4841,    1,  956,    5],
       [   1,  956,    5,  122],
       [ 956,    5,  122,   32],
       [   5,  122,   32,   44],
       [ 122,   32,   44,  548],
       [  32,   44,  548, 2007],
       [  44,  548, 2007, 4842],
       [ 548, 2007, 4842, 1035],
       [2007, 4842, 1035,   13]])

In [8]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [9]:
y = to_categorical(y, num_classes=vocab_size)
y[:1]

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 3, 10)             131360    
_________________________________________________________________
lstm (LSTM)                  (None, 3, 1000)           4044000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense (Dense)                (None, 1000)              1001000   
_________________________________________________________________
dense_1 (Dense)              (None, 13136)             13149136  
Total params: 26,329,496
Trainable params: 26,329,496
Non-trainable params: 0
_________________________________________________________________


In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=3, batch_size=64, callbacks=[checkpoint])

Epoch 1/3
1602/1602 [==============================] - 65s 35ms/step - loss: 6.8757

Epoch 00001: loss improved from inf to 6.87569, saving model to next_words.h5
Epoch 2/3
1602/1602 [==============================] - 57s 35ms/step - loss: 6.3622

Epoch 00002: loss improved from 6.87569 to 6.36219, saving model to next_words.h5
Epoch 3/3
1602/1602 [==============================] - 58s 36ms/step - loss: 5.9949

Epoch 00003: loss improved from 6.36219 to 5.99485, saving model to next_words.h5


In [12]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))


def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [14]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your line: Project Gutenberg's The
['Project', "Gutenberg's", 'The']
man


KeyboardInterrupt: ignored